# Regulizadores

Neste código iremos analisar como funcionam os regularizadores e como eles são usados para evitar *overfitting*, fenônemo que faz com o modelo não generalize bem em outros dataset além do usando durante o treino.

# *Overfitting*

Redes neurais são muito flexíveis porque não se limitam a ver cada atributo a ser aprendido individualmente. Em vez disso, elas podem aprender interações entre os atributos. Por causa disso, mesmo quando temos apenas um pequeno número de atributos, as redes neurais profundas são capazes de chegar ao *overfitting*, um cenário onde o modelo aprende a classificar muito bem (as vezes, perfeitamente) as instâncias de treino, porém não generaliza para outras instâncias não vistas (como são os casos de amostras do conjunto de validação ou teste).

Para evitar esse cenário, algumas técnicas foram propostas para evitar o *overfitting*.

**Como introduzido, nesta aula prática, implementaremos e testaremos duas técnicas para evitar o *overfitting*: *Dropout* e *Weight Decay***

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [1]:
import torch
import torch.nn.functional as F
import torchvision

from torchvision import datasets, transforms
from torch import optim, nn

import numpy as np

import os
import sys
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
n = torch.cuda.device_count()
devices_ids = list(range(n))
device

device(type='mps')

In [2]:
## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [transforms.Resize(resize)]
    transformer += [transforms.ToTensor()]
    transformer = transforms.Compose(transformer)

    mnist_train = datasets.MNIST(root=root, train=True,download=True, transform=transformer)
    mnist_test = datasets.MNIST(root=root, train=False,download=True, transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do Fashion-MNIST
# https://github.com/zalandoresearch/fashion-mnist
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.pytorch', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [transforms.Resize(resize)]
    transformer += [transforms.ToTensor()]
    transformer = transforms.Compose(transformer)

    mnist_train = datasets.FashionMNIST(root=root, train=True, download=True, transform=transformer)
    mnist_test = datasets.FashionMNIST(root=root, train=False, download=True, transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4



    train_iter = torch.utils.data.DataLoader(mnist_train,
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test,
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

In [3]:
# funções básicas
def _get_batch(batch):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.type() != features.type():
        labels = labels.type(features.type())
    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    acc_sum, n, l = torch.Tensor([0]), 0, 0
    
    with torch.no_grad():
      for X, y in data_iter:
          #y = y.astype('float32')
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss,
                   num_epochs):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            trainer.zero_grad()
            l = loss(y_hat, y).sum()
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

# Função para inicializar pesos da rede
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 0.01) # valores iniciais são uma normal
        m.bias.data.fill_(0)

## *Dropout*

*Dropout* é uma das formas mais interessantes de regularizar sua rede neural. 
A ideia do *Droupout* é simples: durante o passo de *Forward*, alguns neurônios são aleatoriamente "desligados", ou seja, são zerados, e não são utilizados em nenhum processamento.
Em cada passo do *Forward*, neurônios diferentes são "desligados" aleatoriamente, de acordo com uma probabilide pré-definida.
Lembrem-se que esse processo só acontece durante o treino.
Durante o teste, *Dropout* não tem nenhuma ação e todos os neurônios são usados para gerar o resultado final.

Formalmente, suponha um neurônio com ativação $h$ e um *Dropout* com probabilide $p$ (de zerar ou "desligar" o neurônio).
Logo, essa técnica irá "desligar" a ativação desse neurônio com probabilidade $p$ ou reescala-la baseado na probabilidade de essa unidade de processamento permanecer ativa (isto é, $1-p$):

$$
\begin{aligned}
h' =
\begin{cases}
    0 & \text{ com probabilidade } p \\
    \frac{h}{1-p} & \text{ caso contrário}
\end{cases}
\end{aligned}
$$

Tal método é interessante e chamou a atenção do mundo acadêmico por ser muito simples de implementar e poder impulsar significativamente o desempenho do modelo.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), para utilizar os benefícios do Dropout basta adicionar a camada homônima (passando como argumento a probabilidade de desligamento dos neurônios) durante a construção da arquitetura.

**Um exemplo é mostrado abaixo utilizando o framework PyTorch.**

Durante o treino, a camada *Dropout* irá "desligar" aleatoriamente algumas saídas da camada anterior (ou equivalentemente, as entradas para a camada subsequente) de acordo com a probabilidade especificada.

Quando o PyTorch não está no modo de treinamento, a camada *Dropout* simplesmente passa os dados sem fazer nenhum "desligamento".

Early stopping & Dropout: não passamos todas features por todos os neurônios em todas as iterações do nosso modelo. Para fazer isso, desativamos alguns neurônios (dropouts) e observamos o impacto disto no training dataset. Existe uma porcentagem ideal de neurônios desativados em que o modelo minimiza o erro, sem overfitting. Se não droparmos nenhum neurônio: overfitting. Se droparmos neurônios demais: o modelo deixa de ser eficiente e começa a fazer classificações erradas. Vídeo bom: https://www.youtube.com/watch?v=aloQiXhNORs 

In [4]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),               # camada densamente conectada
        nn.Dropout(0.2),                   # dropout com 20% de probabilidade de desligar os neurônios
        nn.ReLU(),
        nn.Linear(256, 256),               # camada densamente conectada
        nn.Dropout(0.5),                   # dropout com 50% de probabilidade de desligar os neurônios
        nn.ReLU(),
        nn.Linear(256, 10)                 # camada densamente conectada para classificação
)                     

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr)

# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

100.0%


Extracting /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw



100.0%


Extracting /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw



100.0%


Extracting /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw



100.0%

Extracting /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/danielcarmo/.pytorch/datasets/fashion-mnist/FashionMNIST/raw

training on mps


epoch 1, train loss 1.1802, train acc 0.546, test loss 0.6882, test acc 0.750, time 6.3 sec
epoch 2, train loss 0.5840, train acc 0.782, test loss 0.5647, test acc 0.788, time 3.6 sec
epoch 3, train loss 0.4915, train acc 0.822, test loss 0.4975, test acc 0.815, time 3.6 sec
epoch 4, train loss 0.4489, train acc 0.836, test loss 0.4513, test acc 0.833, time 3.5 sec
epoch 5, train loss 0.4213, train acc 0.846, test loss 0.4486, test acc 0.837, time 3.6 sec
epoch 6, train loss 0.3998, train acc 0.854, test loss 0.4939, test acc 0.816, time 3.5 sec
epoch 7, train loss 0.3825, train acc 0.859, test loss 0.5168, test acc 0.814, time 3.6 sec
epoch 8, train loss 0.3711, train acc 0.864, test loss 0.5306, test acc 0.808, time 3.6 sec
epoch 9, train loss 0.3604, train acc 0.867, test loss 0.4337, test acc 0.840, time 3.6 sec
epoch 10, train loss 0.3482, train acc 0.873, test loss 0.4798, test acc 0.827, time 3.6 sec
epoch 11, train loss 0.3395, train acc 0.873, test loss 0.4803, test acc 0.829,

## *Weight Decay*

*Weight Decay* (comumente chamado regularização *L2*), é uma das técnicas mais utilizadas para regularizar modelos paramétricos de aprendizado de máquina.
A intuição básica por trás do *Weight Decay* é a noção de que entre todas as funções $f$, a função $f=0$ é a mais simples. Intuitivamente, podemos medir funções pela sua proximidade a zero. Mas quão devemos medir a distância entre uma função e zero? Não há uma resposta correta. De fato, ramos inteiros da matemática são dedicados a responder a esta questão.

Para nossos propósitos atuais, uma interpretação muito simples será suficiente: vamos considerar uma função linear $f(\mathbf{x}) = \mathbf{w}^\top \mathbf{x}$ é simples se o seu vetor de peso $\mathbf{w}$ for pequeno. Podemos medir isso via norma $||\mathbf{w}||^2$. Uma maneira de manter o vetor de peso pequeno é adicionar sua norma como um termo de penalidade ao problema de minimizar a função de perda (ou *loss*). Assim, nós substituímos nosso objetivo original, *minimizar o erro de previsão nos rótulos de treinamento*, com novo objetivo, *minimizar o erro de previsão e o termo de penalidade*. Agora, se o vetor de peso se tornar muito grande, nosso algoritmo de aprendizagem vai encontrar mais lucro minimizando a norma $||\mathbf{w}||^2$ do que minimizando o erro de treinamento. 

Tecnicamente, para uma função de custo qualquer $\mathcal{L}$, a adição do novo termo de penalidade (ou *weight decay*) acontece da seguinte forma:

$$\mathcal{L}(\mathbf{w}, b) + \frac{\lambda}{2} \|\boldsymbol{w}\|^2$$

Esse parâmetro não negativo $\lambda \geq 0$ dita a quantidade de regularização. Para $\lambda = 0$, recuperamos nossa função de perda original, enquanto para $\lambda > 0 $ garantimos que os pesos $\mathbf{w}$ não crescerão demais.

### Implementação

Em frameworks atuais (como no MxNet, TensorFlow, e PyTorch), *Weight Decay* pode ser facilmente agregado à função de custo durante a construção do modelo.

**Um exemplo é mostrado abaixo utilizando o framework PyTorch.**

https://www.youtube.com/watch?v=_SlPBbxuqas <- vídeo bom sobre Weight Decay

In [20]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e valor de weight decay
num_epochs, lr, batch_size, weight_decay = 20, 0.5, 256, 0.005

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),               # camada densamente conectada
        nn.ReLU(),
        nn.Linear(256, 256),               # camada densamente conectada
        nn.ReLU(),
        nn.Linear(256, 10)                 # camada densamente conectada para classificação
)                     

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=weight_decay)


# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

training on mps
epoch 1, train loss 1.2811, train acc 0.512, test loss 0.9115, test acc 0.689, time 3.7 sec
epoch 2, train loss 0.6537, train acc 0.755, test loss 0.6474, test acc 0.756, time 3.6 sec
epoch 3, train loss 0.6047, train acc 0.780, test loss 0.7502, test acc 0.746, time 3.6 sec
epoch 4, train loss 0.7718, train acc 0.736, test loss 0.5906, test acc 0.799, time 3.6 sec
epoch 5, train loss 0.6038, train acc 0.782, test loss 0.5031, test acc 0.819, time 3.6 sec
epoch 6, train loss 0.5464, train acc 0.801, test loss 0.6311, test acc 0.754, time 3.6 sec
epoch 7, train loss 0.5440, train acc 0.800, test loss 0.7216, test acc 0.759, time 3.6 sec
epoch 8, train loss 0.5483, train acc 0.801, test loss 0.5547, test acc 0.793, time 3.6 sec
epoch 9, train loss 0.5433, train acc 0.802, test loss 0.4987, test acc 0.820, time 3.7 sec
epoch 10, train loss 1.0658, train acc 0.658, test loss 1.4996, test acc 0.386, time 3.5 sec
epoch 11, train loss 0.8991, train acc 0.652, test loss 0.6865,

## MNIST - *Overfitting* and regularizadores

Agora usaremos um dataset específico juntamente com um rede mais produnda para tentar mostrar o efeito de *overfitting* e entender como as técnicas de regularização aprendidas podem ser usadas para resolver esse problema.

O modelo implementado abaixo usa o dataset MNIST treinando uma rede com quatro camadas que não usa nenhum método de regularização.
Note a diferença entre o *loss* e a acurácia de treino e teste.
Um resultado onde o *loss* do teste é relativamente maior que o treino (caso do exemplo abaixo) indica que um (neste caso, princípio de) *overfitting* está acontecendo.
Use as técnicas vistas nesta aula prática para tratar esse problema.

Especificamente, implemente:

1. Uma versão dessa arquitetura com *Dropout*. Teste diferentes valores de probabilidade de forma a diminuir o *overfitting*.
2. Uma versão desse modelo com *Weight Decay*. Teste diferentes valores de $\lambda$ de forma a diminuir o *overfitting*.
3. Uma versão que combina os dois métodos de regularização aprendidos. Talvez seja necessário testar diferentes valores para a probabilidade do *Dropout* e do $\lambda$.

In [6]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),              
        nn.ReLU(),
        nn.Linear(256, 128),              
        nn.ReLU(),
        nn.Linear(128, 64),              
        nn.ReLU(),
        nn.Linear(64, 10)
)  

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr)

# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

training on mps
epoch 1, train loss 2.3016, train acc 0.111, test loss 2.3010, test acc 0.114, time 3.8 sec
epoch 2, train loss 1.8922, train acc 0.257, test loss 1.1043, test acc 0.579, time 3.6 sec
epoch 3, train loss 0.5050, train acc 0.838, test loss 0.2050, test acc 0.940, time 3.5 sec
epoch 4, train loss 0.1725, train acc 0.950, test loss 0.1447, test acc 0.958, time 3.5 sec
epoch 5, train loss 0.1138, train acc 0.966, test loss 0.1119, test acc 0.966, time 3.5 sec
epoch 6, train loss 0.0861, train acc 0.974, test loss 0.0939, test acc 0.972, time 3.5 sec
epoch 7, train loss 0.0664, train acc 0.980, test loss 0.1169, test acc 0.966, time 3.5 sec
epoch 8, train loss 0.0545, train acc 0.983, test loss 0.1472, test acc 0.956, time 3.5 sec
epoch 9, train loss 0.0435, train acc 0.987, test loss 0.1334, test acc 0.961, time 3.5 sec
epoch 10, train loss 0.3857, train acc 0.916, test loss 0.2103, test acc 0.947, time 3.5 sec
epoch 11, train loss 0.1141, train acc 0.967, test loss 0.1239,

A diferença entre train loss e test loss está grande demais, o que é indicativo do início de um overfitting.

In [24]:
#implementando modelo com Dropout
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), e 
# tamanho do batch
num_epochs, lr, batch_size = 20, 0.5, 256

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),
        nn.Dropout(0.50),    # dropout com 20% de probabilidade de desligar os neurônios          
        nn.ReLU(),
        nn.Linear(256, 128),              
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.Dropout(0.20),   # dropout com 20% de probabilidade de desligar os neurônios           
        nn.ReLU(),
        nn.Linear(64, 10)
)  

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: fashion mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr)

# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)


training on mps
epoch 1, train loss 2.3016, train acc 0.111, test loss 2.3017, test acc 0.103, time 3.8 sec
epoch 2, train loss 2.0023, train acc 0.223, test loss 1.3912, test acc 0.444, time 3.8 sec
epoch 3, train loss 0.8083, train acc 0.720, test loss 0.4889, test acc 0.855, time 3.8 sec
epoch 4, train loss 0.3249, train acc 0.912, test loss 0.3691, test acc 0.898, time 3.8 sec
epoch 5, train loss 0.2291, train acc 0.936, test loss 0.2139, test acc 0.941, time 3.7 sec
epoch 6, train loss 0.1837, train acc 0.948, test loss 0.2841, test acc 0.916, time 3.8 sec
epoch 7, train loss 0.1656, train acc 0.952, test loss 0.1922, test acc 0.945, time 3.7 sec
epoch 8, train loss 0.1492, train acc 0.957, test loss 0.1692, test acc 0.951, time 3.8 sec
epoch 9, train loss 0.1386, train acc 0.959, test loss 0.1573, test acc 0.957, time 3.8 sec
epoch 10, train loss 0.1329, train acc 0.961, test loss 0.1815, test acc 0.950, time 3.7 sec
epoch 11, train loss 0.1243, train acc 0.964, test loss 0.1471,

Um resultado razoável, melhor do que sem o uso de dropout!

In [22]:
#implementação usando weight decay
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e valor de weight decay
num_epochs, lr, batch_size, weight_decay = 20, 0.5, 256, 0.005

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),               # camada densamente conectada
        nn.ReLU(),
        nn.Linear(256, 256),               # camada densamente conectada
        nn.ReLU(),
        nn.Linear(256, 10)                 # camada densamente conectada para classificação
)                     

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=weight_decay)


# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

training on mps
epoch 1, train loss 1.0213, train acc 0.647, test loss 0.4307, test acc 0.871, time 3.6 sec
epoch 2, train loss 0.2929, train acc 0.914, test loss 0.2638, test acc 0.922, time 3.6 sec
epoch 3, train loss 0.2170, train acc 0.937, test loss 0.8225, test acc 0.837, time 3.5 sec
epoch 4, train loss 0.2037, train acc 0.942, test loss 0.1826, test acc 0.946, time 3.6 sec
epoch 5, train loss 0.1900, train acc 0.947, test loss 0.2142, test acc 0.936, time 3.5 sec
epoch 6, train loss 0.2044, train acc 0.943, test loss 0.1697, test acc 0.953, time 3.6 sec
epoch 7, train loss 0.1800, train acc 0.951, test loss 0.1625, test acc 0.953, time 3.6 sec
epoch 8, train loss 0.2076, train acc 0.943, test loss 0.2002, test acc 0.936, time 3.6 sec
epoch 9, train loss 0.1709, train acc 0.953, test loss 0.1790, test acc 0.954, time 3.5 sec
epoch 10, train loss 0.1566, train acc 0.957, test loss 0.2024, test acc 0.936, time 3.6 sec
epoch 11, train loss 0.1621, train acc 0.955, test loss 0.2034,

In [26]:
#implementação usando weight decay e dropout
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e valor de weight decay
num_epochs, lr, batch_size, weight_decay = 20, 0.5, 256, 0.01

# rede simples somente com perceptrons e camadas densamente conectadas
net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(784, 256),               # camada densamente conectada
        nn.Dropout(0.20),    # dropout com 20% de probabilidade de desligar os neurônios
        nn.ReLU(),
        nn.Linear(256, 256),               # camada densamente conectada
        nn.Dropout(0.20),    # dropout com 20% de probabilidade de desligar os neurônios
        nn.ReLU(),
        nn.Linear(256, 10)                 # camada densamente conectada para classificação
)                     

net.apply(weights_init)
net.to(device) # diz para a rede que ela deve ser treinada na GPU

# função de custo (ou loss)
loss = nn.CrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_mnist(batch_size)

# trainer do gluon
trainer = optim.SGD(net.parameters(), lr=lr, weight_decay=weight_decay)


# treinamento e validação
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)


training on mps
epoch 1, train loss 1.1933, train acc 0.576, test loss 0.5532, test acc 0.820, time 3.8 sec
epoch 2, train loss 0.4024, train acc 0.881, test loss 0.4138, test acc 0.866, time 3.8 sec
epoch 3, train loss 0.3464, train acc 0.899, test loss 0.5107, test acc 0.831, time 3.7 sec
epoch 4, train loss 0.3204, train acc 0.910, test loss 0.4552, test acc 0.859, time 3.7 sec
epoch 5, train loss 0.3108, train acc 0.912, test loss 0.3162, test acc 0.912, time 3.8 sec
epoch 6, train loss 0.3068, train acc 0.913, test loss 0.3049, test acc 0.906, time 3.7 sec
epoch 7, train loss 0.2981, train acc 0.915, test loss 0.3133, test acc 0.906, time 3.7 sec
epoch 8, train loss 0.2965, train acc 0.916, test loss 0.2416, test acc 0.931, time 3.7 sec
epoch 9, train loss 0.2999, train acc 0.916, test loss 0.2926, test acc 0.913, time 3.7 sec
epoch 10, train loss 0.2869, train acc 0.920, test loss 0.2484, test acc 0.927, time 3.7 sec
epoch 11, train loss 0.2918, train acc 0.918, test loss 0.2380,

## Exercícios

1. Qual impacto de alterar a probabilidade da camada de *Dropout*?  Como fica uma rede com probabilidade de 50% de "desligar" os neurônios?
1. E em relação ao valor de $\lambda$ para o *Weight Decay*, qual impacto? Teste valores como 0.0005 e 0.0001.
1. Qual efeito da quantidade de *epochs* na acurácia final do modelo?


1 - Se aumentarmos demais a probabilidade de dropout dos neurônios, o modelo pode ficar generalizado demais, não conseguindo prever os inputs corretamente em nenhum caso. No entanto, deve haver uma probabilidade de dropout ideal, em que o overfitting é previnido e a precisão do modelo continua satisfatória. 
2 - Aparentemente, após a realização de testes, percebemos que o aumento do lambda para um weight decay faz com que sejam exigidas mais épocas de treinamento para que o modelo seja treinado com eficácian alta. Isto se dá pois quanto maior for lambda, mais distante do erro real estamos penalizando nosso modelo, o que faz com que o modelo se sinta cada vez mais impreciso, demorando mais para aprender. 
3 - Quanto mais épocas, mais o nosso modelo aprende e, potencialmente, tem seu overfitting aumentado. 

